In [3]:
import pandas as pd,re, pickle, scipy
from scipy.sparse import csr_matrix
from sklearn.naive_bayes import *
import numpy as np
from sklearn.feature_selection import *
from sklearn.feature_extraction import *
from sklearn.feature_extraction.text import *
from sklearn.preprocessing import LabelEncoder as LE
from sklearn.preprocessing import *
from sklearn.decomposition import PCA
from sklearn.metrics import *
from sklearn.kernel_ridge import *
from sklearn import tree
from sklearn.ensemble import *
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.svm import LinearSVC, SVC, OneClassSVM
from sklearn.model_selection import *
from sklearn.linear_model import *
from collections import defaultdict

from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE, ADASYN
from url_utils import URLUtils
from keras.layers import *
from keras.models import  *
from keras.utils import to_categorical
from keras.metrics import *
from encoder import Encoder
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [4]:
df = pd.read_csv('/Users/bjayaram/Documents/Kaggle/digits/train.csv')


In [15]:
# data_columns = filter(lambda x: x.startswith('pixel') and df[x].sum() != 0,df.columns)
data_columns = filter(lambda x: x.startswith('pixel'), df.columns)
label_columns = 'label'
print len(data_columns), len(df.columns)


784 785


label         9
pixel0        0
pixel1        0
pixel2        0
pixel3        0
pixel4        0
pixel5        0
pixel6        0
pixel7        0
pixel8        0
pixel9        0
pixel10       0
pixel11       0
pixel12     116
pixel13     254
pixel14     216
pixel15       9
pixel16       0
pixel17       0
pixel18       0
pixel19       0
pixel20       0
pixel21       0
pixel22       0
pixel23       0
pixel24       0
pixel25       0
pixel26       0
pixel27       0
pixel28       0
           ... 
pixel754      0
pixel755      0
pixel756      0
pixel757      0
pixel758      0
pixel759      0
pixel760      0
pixel761    177
pixel762    231
pixel763    253
pixel764    254
pixel765    254
pixel766    255
pixel767    255
pixel768    255
pixel769    255
pixel770    255
pixel771    255
pixel772    255
pixel773    255
pixel774    254
pixel775    254
pixel776    253
pixel777    253
pixel778    254
pixel779     62
pixel780      0
pixel781      0
pixel782      0
pixel783      0
Length: 785, dtype: int6

In [30]:
X = df[data_columns].values
Y = df[label_columns].values
X=MinMaxScaler().fit_transform(X,Y)

X.shape,Y.shape
df[label_columns].value_counts()


(60000, 28, 28)


In [69]:


# Plot ad hoc mnist instances
from keras.datasets import mnist
import matplotlib.pyplot as plt
# load (downloaded if needed) the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data('/Users/bjayaram/Documents/Kaggle/digits/mnist.npz')
# plot 4 images as gray scale
plt.subplot(221)
plt.imshow(X_train[0], cmap=plt.get_cmap('gray'))
print X_train.shape

(60000, 28, 28)


In [ ]:
# model = GridSearchCV(RandomForestClassifier(), cv = 5, n_jobs= -1, verbose=5, 
#                            param_grid={'n_estimators':[100], 'class_weight': ['balanced']})
model = RandomizedSearchCV(SVC(), scoring='f1',verbose=5, n_jobs= -1, 
                           param_distributions={'C':range(1,100,10), 
                           'kernel':['poly','rbf','linear'],'gamma':map(lambda x:10**x, range(-3,3))})
# model = MLPClassifier(hidden_layer_sizes=(1000,200))
model.fit(X,Y )
model.best_estimator_
model.best_score_

In [ ]:
print "Starting model generation"
model = Sequential()
num_pixels=X.shape[1]
model.add(Dense(num_pixels, input_shape=(num_pixels,), kernel_initializer='normal', activation='relu'))
model.add(Dense(100, kernel_initializer='normal', activation='relu'))
model.add(Dense(10, kernel_initializer='normal', activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy',mae])

# model.add(Dense(num_pixels+10, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
# model.add(Dense(10, kernel_initializer='normal', activation='softmax'))
# 	
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',mae])


ValueError: Error when checking input: expected conv2d_3_input to have 4 dimensions, but got array with shape (42000, 784)

In [56]:

Y1=to_categorical(Y)

model.fit(X,Y1, verbose=2, validation_split=0.2, shuffle=True, epochs=10)
model.evaluate(X,Y1)


ValueError: Error when checking input: expected conv2d_3_input to have 4 dimensions, but got array with shape (42000, 784)

In [84]:
df = pd.read_csv('/Users/bjayaram/Documents/Kaggle/digits/test.csv')
X_test = MinMaxScaler().fit_transform(df[data_columns].values)

X_test.shape

(28000, 784)

In [80]:
y_test1=model.predict(X_test)
y_test = np.argmax(y_test1, axis = 1)
print y_test
res= pd.DataFrame()

res['ImageId']=range(1,1+len(y_test))
res['Label']=y_test
res.to_csv('/Users/bjayaram/Documents/Kaggle/digits/results.csv', index=None)

ValueError: Error when checking : expected conv2d_3_input to have 4 dimensions, but got array with shape (10000, 28, 28)

In [52]:
 #CNN
 

import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

In [87]:

# create model
model = Sequential()
model.add(Conv2D(32, (8, 8), input_shape=(1, 28, 28), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [89]:
X2d = np.rollaxis(np.dstack(map(lambda x:np.reshape(x, (28,28)), X)),2)
X2d.shape

X=X.reshape(X2d.shape[0], 1, 28, 28).astype('float32')

Y1=to_categorical(Y)

model.fit(X,Y1, verbose=2, validation_split=0.2, shuffle=True, epochs=10)
model.evaluate(X,Y1)

Train on 33600 samples, validate on 8400 samples
Epoch 1/10


 - 53s - loss: 0.1829 - acc: 0.9445 - val_loss: 0.0772 - val_acc: 0.9764


Epoch 2/10


 - 60s - loss: 0.0630 - acc: 0.9806 - val_loss: 0.0572 - val_acc: 0.9814


Epoch 3/10


...

41856/42000 [============================>.] - ETA: 0s

41920/42000 [============================>.] - ETA: 0s

41984/42000 [============================>.] - ETA: 0s

42000/42000 [==============================] - 34s 818us/step


[0.015483909915761701, 0.99592857142857139]

In [90]:
X2d = np.rollaxis(np.dstack(map(lambda x:np.reshape(x, (28,28)), X_test)),2)
print X2d.shape

X_test=X2d.reshape(X2d.shape[0], 1, 28, 28).astype('float32')

y_test1=model.predict(X_test)
y_test = np.argmax(y_test1, axis = 1)

res= pd.DataFrame()

res['ImageId']=range(1,1+len(y_test))
res['Label']=y_test
res.to_csv('/Users/bjayaram/Documents/Kaggle/digits/results.csv', index=None)

(28000, 28, 28)


In [90]:
print y_test, len(y_test)

[2 0 9 ..., 3 9 2] 28000
